In [195]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
from pathlib import Path
from skimage.feature import hog
from skimage.feature import local_binary_pattern

In [11]:
p = Path('.')
dataset = p / 'dataset'
features = p / 'features'

In [3]:
def global_color_histogram_save_feature(img_path, save_path):
    img = cv2.imread(img_path.as_posix())

    ub = 256
    hist = np.concatenate([cv2.calcHist([img], [i], None, [ub], [0, ub])[:, 0] for i in range(3)])
    hist /= hist.sum()
    np.save(save_path, hist)

In [4]:
def global_color_histogram_gray_save_feature(img_path, save_path):
    img = cv2.imread(img_path.as_posix(), cv2.IMREAD_GRAYSCALE)

    ub = 256
    hist = cv2.calcHist([img], [0], None, [ub], [0, ub])[:, 0]
    hist /= hist.sum()
    np.save(save_path, hist)

In [5]:
def regional_color_histogram_save_feature(img_path, save_path):
    img = cv2.imread(img_path.as_posix())

    ub = 256
    h, w, _ = img.shape
    h //= 2
    w //= 2
    hists = []
    for i in range(2):
        for j in range(2):
            region = img[h*i:h*(i+1), w*j:w*(j+1), :]
            hist = np.concatenate([cv2.calcHist([region], [i], None, [ub], [0, ub])[:, 0] for i in range(3)])
            hists.append(hist)
    hist = np.concatenate(hists)
    hist /= hist.sum()
    np.save(save_path, hist)

In [63]:
def sift_nfeatures500_save_feature(img_path, save_path):
    img = cv2.imread(img_path.as_posix())
    sift = cv2.xfeatures2d.SIFT_create(nfeatures=500)
    _, des = sift.detectAndCompute(img, None)
    np.save(save_path, des)

In [135]:
def surf_hessianThreshold2000_save_feature(img_path, save_path):
    img = cv2.imread(img_path.as_posix())
    surf = cv2.xfeatures2d.SURF_create(hessianThreshold=2000)
    _, des = surf.detectAndCompute(img, None)
    np.save(save_path, des)

In [241]:
def hog_save_feature(img_path, save_path):
    img = cv2.imread(img_path.as_posix())
    des = hog(img, orientations=8, pixels_per_cell=(img.shape[0] // 16, img.shape[1] // 16),
              cells_per_block=(1, 1), block_norm='L2-Hys', multichannel=True)
    np.save(save_path, des)

In [ ]:
def local_binary_pattern_save_feature(img_path, save_path):
    img = cv2.imread(img_path.as_posix(), cv2.IMREAD_GRAYSCALE)
    radius = 2
    n_points = 8 * radius
    METHOD = 'uniform'
    lbp = local_binary_pattern(img, n_points, radius, METHOD)
    lbpi = np.uint8(lbp)
    hist = cv2.calcHist([lbpi], [0], None, [n_points + 2], [0, n_points + 2])[:, 0]
    hist /= hist.sum()
    np.save(save_path, hist)

In [242]:
feature_name2func = {
    'global_color_histogram': global_color_histogram_save_feature,
    'global_color_histogram_gray': global_color_histogram_gray_save_feature,
    'regional_color_histogram': regional_color_histogram_save_feature,
    'sift_nfeatures500': sift_nfeatures500_save_feature,
    'surf_hessianThreshold2000': surf_hessianThreshold2000_save_feature,
    'local_binary_pattern': local_binary_pattern_save_feature,
    'hog': hog_save_feature,
}

In [243]:
feature_name = 'hog'
save_feature = feature_name2func[feature_name]
for category in dataset.iterdir():
    for typ in category.iterdir():
        save_dir = features / feature_name / category.name / typ.name
        save_dir.mkdir(parents=True, exist_ok=True)
        for img_path in typ.iterdir():
            save_feature(img_path, save_dir / img_path.name)
    print(category.name, 'done')
print('done')

dvd_covers done
video_frames done
business_cards done
museum_paintings done
cd_covers done
book_covers done
done
CPU times: user 42min 16s, sys: 3min 59s, total: 46min 16s
Wall time: 46min 16s


In [254]:
cat_num2id = dict()
refs = []
for category in (features / feature_name).iterdir():
    ref = category / 'Reference'
    print(ref)
    for npy in ref.iterdir():
        cat_num2id[category.name + npy.stem] = len(refs)
        refs.append(np.float32(np.load(npy)))

features/hog/dvd_covers/Reference
features/hog/video_frames/Reference
features/hog/business_cards/Reference
features/hog/museum_paintings/Reference
features/hog/cd_covers/Reference
features/hog/book_covers/Reference


In [247]:
refs[-1].shape

(2048,)

In [255]:
def run_one_new(feature_dirname, method, descending=False):
    for category in (features / feature_dirname).iterdir():
        num_qrys = len(list(category.rglob('*npy')))
        s_at_1 = 0
        s_at_5 = 0
        for typ in category.glob('[!R]*'):
            for npy in typ.iterdir():
                qry = np.float32(np.load(npy))
                dists = [cv2.compareHist(qry, ref, method) for ref in refs]
                indices = np.array(dists).argsort()
                if descending:
                    indices = indices[::-1]
                top5 = indices[:5]
                s_at_5 += cat_num2id[category.name + npy.stem] in top5
                s_at_1 += cat_num2id[category.name + npy.stem] == top5[0]
        print('{},{},{}'.format(category.name, s_at_1 / num_qrys, s_at_5 / num_qrys), end=',')
    print()

In [19]:
print(cv2.HISTCMP_CORREL)
print(cv2.HISTCMP_CHISQR) # good?
print(cv2.HISTCMP_INTERSECT) # good?
print(cv2.HISTCMP_BHATTACHARYYA)
print(cv2.HISTCMP_CHISQR_ALT)
descendings = [True, False, True, False, False]

0
1
2
3
4


In [256]:
for i in range(5):
    run_one_new('hog', i, descendings[i])

dvd_covers,0.0,0.004,video_frames,0.064,0.118,business_cards,0.032,0.074,museum_paintings,0.041758241758241756,0.07032967032967033,cd_covers,0.01,0.026,book_covers,0.011881188118811881,0.033663366336633666,
dvd_covers,0.0,0.0,video_frames,0.038,0.06,business_cards,0.002,0.008,museum_paintings,0.01098901098901099,0.05714285714285714,cd_covers,0.0,0.0,book_covers,0.0019801980198019802,0.007920792079207921,
dvd_covers,0.0,0.0,video_frames,0.0,0.0,business_cards,0.0,0.0,museum_paintings,0.017582417582417582,0.046153846153846156,cd_covers,0.0,0.0,book_covers,0.0,0.0,
dvd_covers,0.0,0.0,video_frames,0.002,0.002,business_cards,0.0,0.0,museum_paintings,0.008791208791208791,0.04835164835164835,cd_covers,0.0,0.0,book_covers,0.0019801980198019802,0.0019801980198019802,
dvd_covers,0.0,0.0,video_frames,0.006,0.008,business_cards,0.0,0.0,museum_paintings,0.008791208791208791,0.05054945054945055,cd_covers,0.0,0.0,book_covers,0.0019801980198019802,0.0019801980198019802,


In [175]:
def test_algo(create):
    for i in range(1, 10):
        img = cv2.imread((dataset / 'book_covers/5800/00{}.jpg'.format(i)).as_posix())
        algo = create()
        kp, des = algo.detectAndCompute(img, None)
        print(img.shape, len(kp), des.shape)

In [138]:
cat_num2id = dict()
matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_FLANNBASED)
for category in (features / feature_name).iterdir():
    ref = category / 'Reference'
    for npy in ref.iterdir():
        cat_num2id[category.name + npy.stem] = len(cat_num2id)
        matcher.add([np.load(npy)])
    print(ref, 'done')

features/surf_hessianThreshold2000/dvd_covers/Reference done
features/surf_hessianThreshold2000/video_frames/Reference done
features/surf_hessianThreshold2000/business_cards/Reference done
features/surf_hessianThreshold2000/museum_paintings/Reference done
features/surf_hessianThreshold2000/cd_covers/Reference done
features/surf_hessianThreshold2000/book_covers/Reference done


In [168]:
def run_one_flann(feature_dirname):
    for category in (features / feature_dirname).iterdir():
        num_qrys = len(list(category.rglob('*npy')))
        s_at_1 = 0
        s_at_5 = 0
        for typ in category.glob('[!R]*'):
            for npy in typ.iterdir():
                qry = np.load(npy)
                try:
                    matches = matcher.knnMatch(qry, k=2)
                except:
                    continue
                ratio = 0.8
                good_matches = [m1 for m1, m2 in matches if m1.distance < ratio * m2.distance]
                if len(good_matches) == 0:
                    good_matches = [m1 for m1, m2 in matches]
                indices = np.bincount([m.imgIdx for m in good_matches]).argsort()[::-1]
                top5 = indices[:5]
                s_at_5 += cat_num2id[category.name + npy.stem] in top5
                s_at_1 += cat_num2id[category.name + npy.stem] == top5[0]
        print('{},{},{}'.format(category.name, s_at_1 / num_qrys, s_at_5 / num_qrys), end=',')
    print()

In [169]:
run_one_flann('surf_hessianThreshold2000')

dvd_covers,0.75,0.776,video_frames,0.744,0.772,business_cards,0.522,0.594,museum_paintings,0.5340659340659341,0.5824175824175825,cd_covers,0.658,0.704,book_covers,0.6633663366336634,0.7425742574257426,
CPU times: user 1min, sys: 128 ms, total: 1min 1s
Wall time: 1min 1s
